<a id='top'></a>

In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


# %matplotlib nbagg 
%matplotlib notebook
# %matplotlib inline



## Third party 
import numpy as np
import os, time, zarr, sys
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl

import unslice.IO as io
from unslice.utils import *
from unslice.registration.featmatch import *
from unslice.registration.transform import *
from unslice.registration.rigid import *
from unslice.registration.gpu_transform import *
from unslice.registration.utils import *
from unslice.segmentation import *
from unslice.tracing.pyoof import OOF, apply_oof_v2
from unslice.tracing.skel import *
from unslice.flatten import *
from unslice.lightsheetcorrect import *




In [2]:
# Parameters that are constant throughout notebook
working_dir = '/mnt/share3/webster/mEhmAD_2-3'

def bdir(fname):
    return os.path.join(working_dir, fname)

# prefix to add to the beginning of each filename 
name_prefix = '3-abeta_cropped' 
name_prefix2 = '2-abeta_cropped'

# Table of contents

### Pre-processing
[1. Convert to zarr](#convert)<br>
[2. Flatten warp](#flattenwarp)<br>
[3. Ptau warp](#anchorwarp)<br>

# Convert to zarr
<a id='convert'></a>

[Return to top](#top)

### slab 3

In [6]:
# First have to convert R1 lecwarp into flattened, then flattened into original

xrange = [5759,8410]
yrange = [5981,7328]
zrange = [6300,8300]
warped_zarr_path = '/mnt/share3/webster/mEhmAD_1-3_real/3-ptau_flattened_anchorwarp_r1.zarr'
grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/grid_anchor_tps_r1_upsampled.npy'


# We can find true "bounding box" by warping a bunch of points 
num_pts = 8
X,Y,Z = np.meshgrid(np.linspace(xrange[0],xrange[1],num_pts),
           np.linspace(yrange[0],yrange[1],num_pts),
           np.linspace(zrange[0],zrange[1],num_pts))

grid_pts = np.vstack((X.ravel(),Y.ravel(),Z.ravel())).T
bbox = grid_transform_pts(grid_path, grid_pts, warped_zarr_path, save_path=None, save_json=False, inverse_transform=False)

In [7]:
# Below are the coordinates in the flattened frame

print(bbox[:,0].min(),bbox[:,0].max())
print(bbox[:,1].min(),bbox[:,1].max())
print(bbox[:,2].min(),bbox[:,2].max())

4790.654015139719 7711.115582936718
5819.512784971864 7404.851941723634
2486.355667152383 4662.816999972807


In [8]:
# Now we transform these coordinates into flattened frame

xrange = [4790,7711]
yrange = [5819,7405]
zrange = [2486,4663]
warped_zarr_path = '/mnt/share3/webster/mEhmAD_1-3_real/3-ptau_flattened.zarr'
grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/3-lec_grid_flatten_upsampled.npy'

# We can find true "bounding box" by warping a bunch of points 
num_pts = 8
X,Y,Z = np.meshgrid(np.linspace(xrange[0],xrange[1],num_pts),
           np.linspace(yrange[0],yrange[1],num_pts),
           np.linspace(zrange[0],zrange[1],num_pts))

grid_pts = np.vstack((X.ravel(),Y.ravel(),Z.ravel())).T
bbox = grid_transform_pts(grid_path, grid_pts, warped_zarr_path, save_path=None, save_json=False, inverse_transform=False)

In [9]:
# Below are the coordinates in the original frame

print(bbox[:,0].min(),bbox[:,0].max())
print(bbox[:,1].min(),bbox[:,1].max())
print(bbox[:,2].min(),bbox[:,2].max())

4814.70244594863 7710.463330216568
5733.991739695085 7349.598763739279
1874.2116410280928 4483.7374793936


In [10]:
# 3

slab_path = '/mnt/cephfs/general/juhyuk/15xdata/20220516_22_11_41_mEhmAD_3_abeta.lec.ptau/Ex_488_Em_1_destriped_stitched_tiffs'
slab_zarr_path = bdir(name_prefix+'.zarr')
pc2_img_size = (21268,9755,5918) #(22177,7817,5933) this is the mehmad2 size


## Optional parameters 
load_num_slices = 40 # should be smaller than z chunk size, default None 
resample_num_slices = 1 # number of slices to be processed in one go for resampling, default 1
resample_factor = (1,1) # original is (0.306,0.433,0.306)
chunks = (200,200,200) 
num_workers = 20 
file_names = 'img_[0-9]{4}.tiff' # default 'img_[0-9]{4}.tiff'

## crop
coord0s = [2740,1096,862] # this is how much to add to coordinates 
# This is the range in the unflattened image, but cropped from original (calculated above)
xrange = [4814,7711]; xrange = [xrange[k]+coord0s[0] for k in range(2)]
yrange = [5733,7350]; yrange = [yrange[k]+coord0s[1] for k in range(2)]
zrange = [1874,4484]; zrange = [zrange[k]+coord0s[2] for k in range(2)]

## rotate
lateral_rotate_angle = None # default None

## flip
flip = (0,0,0) # default (0,0,0)


#############
start = time.time()
utils.convert_to_zarr_v2(slab_path, slab_zarr_path, pc2_img_size, load_num_slices=load_num_slices,
                        resample_num_slices=resample_num_slices, file_names=file_names, 
                        chunks=chunks, num_workers=num_workers, lateral_rotate_angle=lateral_rotate_angle,
                        flip=flip, crop_xcoords=xrange, crop_ycoords=yrange, crop_zcoords=zrange,
                        resample_factor=resample_factor)
print(time.time()-start, 'seconds elapsed')

Processing chunk x:7554-10451, y:6829-8446, z:2736-2776


100%|██████████| 40/40 [00:51<00:00,  1.30s/it]

Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...



100%|██████████| 135/135 [00:02<00:00, 56.20it/s]

Processing chunk x:7554-10451, y:6829-8446, z:2776-2816


...

Processing chunk x:7554-10451, y:6829-8446, z:5336-5346



100%|██████████| 10/10 [00:12<00:00,  1.20s/it]

Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...



100%|██████████| 135/135 [00:01<00:00, 106.34it/s]

4029.192533016205 seconds elapsed


## slab 2

In [6]:
# FIgure out where to crop

xrange = [4545,7353]
yrange = [5226,6718]
zrange = [0,2000]
warped_zarr_path = '/mnt/share3/webster/mEhmAD_2-3/2-ptau_flattened.zarr'
grid_path = '/mnt/share3/webster/mEhmAD_2-3/warping_grids/2-lec_grid_flatten_upsampled.npy'


# We can find true "bounding box" by warping a bunch of points 
num_pts = 8
X,Y,Z = np.meshgrid(np.linspace(xrange[0],xrange[1],num_pts),
           np.linspace(yrange[0],yrange[1],num_pts),
           np.linspace(zrange[0],zrange[1],num_pts))

grid_pts = np.vstack((X.ravel(),Y.ravel(),Z.ravel())).T
bbox = grid_transform_pts(grid_path, grid_pts, warped_zarr_path, save_path=None, save_json=False, inverse_transform=False)

In [7]:
# Below are the coordinates in the original non-flattened field of view 

print(bbox[:,0].min(),bbox[:,0].max())
print(bbox[:,1].min(),bbox[:,1].max())
print(bbox[:,2].min(),bbox[:,2].max())

4554.60198294871 7419.1987282669115
5177.535863891769 6741.2023181141585
-381.4675662321031 1939.144180790585


In [8]:
# 2

slab_path = '/mnt/cephfs/general/juhyuk/15xdata/20220512_21_26_12_mEhmAD_2_abeta.lec.ptau/Ex_488_Em_1_destriped_stitched_tiffs'
slab_zarr_path = bdir(name_prefix2+'.zarr')
pc2_img_size = (22177,7817,5933)


## Optional parameters 
load_num_slices = 40 # should be smaller than z chunk size, default None 
resample_num_slices = 1 # number of slices to be processed in one go for resampling, default 1
resample_factor = (1,1) # original is (0.306,0.433,0.306)
chunks = (200,200,200) 
num_workers = 20 
file_names = 'img_[0-9]{4}.tiff' # default 'img_[0-9]{4}.tiff'

## crop
coord0s = [3600,0,553] # this is how much to add to coordinates 
# This is the range in the unflattened image, but cropped from original (calculated above)
xrange = [4554,7420]; xrange = [xrange[k]+coord0s[0] for k in range(2)]
yrange = [5177,6742]; yrange = [yrange[k]+coord0s[1] for k in range(2)]
zrange = [0,1940]; zrange = [zrange[k]+coord0s[2] for k in range(2)]

## rotate
lateral_rotate_angle = None # default None

## flip
flip = (0,0,0) # default (0,0,0)


#############
start = time.time()
utils.convert_to_zarr_v2(slab_path, slab_zarr_path, pc2_img_size, load_num_slices=load_num_slices,
                        resample_num_slices=resample_num_slices, file_names=file_names, 
                        chunks=chunks, num_workers=num_workers, lateral_rotate_angle=lateral_rotate_angle,
                        flip=flip, crop_xcoords=xrange, crop_ycoords=yrange, crop_zcoords=zrange,
                        resample_factor=resample_factor)
print(time.time()-start, 'seconds elapsed')

Processing chunk x:8154-11020, y:5177-6742, z:553-593


100%|██████████| 40/40 [00:38<00:00,  1.04it/s]


Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...


100%|██████████| 120/120 [00:02<00:00, 54.86it/s]


Processing chunk x:8154-11020, y:5177-6742, z:593-633


...

Processing chunk x:8154-11020, y:5177-6742, z:2473-2493


100%|██████████| 20/20 [00:24<00:00,  1.23s/it]


Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...


100%|██████████| 120/120 [00:12<00:00,  9.50it/s]


2792.0615351200104 seconds elapsed


## Flatten warp the slabs

[Return to top](#top)

<a id='flattenwarp'></a>

## Crop flattening grids

### (3)

In [14]:
# (3)
grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/3-lec_grid_flatten_upsampled.npy'
original_shape = (16172,8004,5052)
new_grid_spacing = 3*(64,) # doesn't matter if range_mode= 'moving'
range_mode='moving'

# We will compute  the grid spacing from the zarr specifically 
save_grid_path = '/mnt/share3/webster/mEhmAD_2-3/warping_grids/3-lec_grid_flatten_upsampled_cropped_abeta.npy'

# these are the coordinates in original image of FOV
xrange = [4814,7711]
yrange = [5733,7350]
zrange = [1874,4484]
################
grid_new,new_shape = crop_grid(grid_path, original_shape, new_grid_spacing, range_mode=range_mode, 
                         xrange=xrange, yrange=yrange, zrange=zrange, save_grid_path=save_grid_path)

New fixed image shape: (3093, 1808, 2953)
New fixed range: x 4703 7796
New fixed range: y 5744 7552
New fixed range: z 2099 5052


In [21]:
# 3
fixed_zarr_path = new_shape
moving_zarr_path = bdir(name_prefix+'.zarr')
warped_zarr_path = bdir(name_prefix+'_flattened.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(64,)
chunks=3*(200,)
nb_workers = 8


# grid I/O 
save_grid_values_path = None
use_grid_values_path = '/mnt/share3/webster/mEhmAD_2-3/warping_grids/3-lec_grid_flatten_upsampled_cropped_abeta.npy'




##########################

moving_pts_paths = None # doesn't actually matter for this 
fixed_pts_paths =  None

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None
zadd = fixed_zarr_path[2]-zarr.open(moving_zarr_path).shape[2]

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,zadd=zadd,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
# tiff_path = warped_zarr_path[:-5]+'_tiffs'
# convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(3093, 1808, 2953)
Loading grid values...
Warping image...
Moving image size: 24.45282378 GB


100%|██████████| 2400/2400 [04:04<00:00,  9.83it/s]


Time elapsed: 7.714502 minutes


In [3]:
# (3) Now we lectin warp it as well 
grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/grid_anchor_tps_r1_upsampled.npy'
original_shape = (18100,8100,8732)
new_grid_spacing = 3*(64,) # doesn't matter if range_mode= 'moving'
range_mode='moving'

# We will compute  the grid spacing from the zarr specifically 
save_grid_path = '/mnt/share3/webster/mEhmAD_2-3/warping_grids/3-grid_anchor_tps_r1_upsampled_cropped_abeta.npy'

# these are the coordinates in flattened image FOV 
# computed when warping image earlier 
xrange = [4703,7796]
yrange = [5744,7552]
zrange = [2099,5052]


################
grid_new,new_shape = crop_grid(grid_path, original_shape, new_grid_spacing, range_mode=range_mode, 
                         xrange=xrange, yrange=yrange, zrange=zrange, save_grid_path=save_grid_path)

New fixed image shape: (3478, 2139, 2781)
New fixed range: x 5346 8824
New fixed range: y 5637 7776
New fixed range: z 5886 8667


### Coordinates
New fixed range: x 5346 8824 <br>
New fixed range: y 5637 7776<br>
New fixed range: z 5886 8667<br>


### coordinates of the PTAU cropped image
xrange = [5759,8410]<br>
yrange = [5981,7328]<br>
zrange = [6300,8300]

In [5]:
# 3
fixed_zarr_path = new_shape
moving_zarr_path = bdir(name_prefix+'_flattened.zarr')
warped_zarr_path = bdir(name_prefix+'_flattened_lectinwarp.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(64,)
chunks=3*(200,)
nb_workers = 8


# grid I/O 
save_grid_values_path = None
use_grid_values_path = '/mnt/share3/webster/mEhmAD_2-3/warping_grids/3-grid_anchor_tps_r1_upsampled_cropped_abeta.npy'




##########################

moving_pts_paths = None # doesn't actually matter for this 
fixed_pts_paths =  None

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None
zadd = fixed_zarr_path[2]-zarr.open(moving_zarr_path).shape[2]

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,zadd=zadd,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
# tiff_path = warped_zarr_path[:-5]+'_tiffs'
# convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(3478, 2139, 2781)
Loading grid values...
Warping image...
Moving image size: 33.027202464 GB


100%|██████████| 2772/2772 [04:22<00:00, 22.61it/s]


Time elapsed: 7.567562 minutes


In [6]:
# recrop image to be correct size 

# We need to crop the flattened to be the same size as the ptau flattened zarr to be warped
    
# bounding box of moving in flat frame
xrange = [5759-5346,8410-5346]
yrange = [5981-5637,7328-5637]
zrange = [6300-5886,8300-5886]
num_workers = 24
source_zarr_path = bdir(name_prefix+'_flattened_lectinwarp.zarr')
sink_zarr_path = bdir(name_prefix+'_flattened_lectinwarp_recropped.zarr')



###############
crop_zarr(source_zarr_path, sink_zarr_path, xrange=xrange, yrange=yrange, zrange=zrange, 
          load_num_slices=None, num_workers=num_workers)

Processing chunk x:413-3064, y:344-1691, z:414-614


100%|██████████| 98/98 [00:06<00:00, 15.94it/s]


Processing chunk x:413-3064, y:344-1691, z:614-814


100%|██████████| 98/98 [00:06<00:00, 15.97it/s]


Processing chunk x:413-3064, y:344-1691, z:814-1014


...

100%|██████████| 98/98 [00:06<00:00, 15.18it/s]


Processing chunk x:413-3064, y:344-1691, z:2014-2214


100%|██████████| 98/98 [00:05<00:00, 18.55it/s]


Processing chunk x:413-3064, y:344-1691, z:2214-2414


100%|██████████| 98/98 [00:05<00:00, 18.31it/s]


In [7]:
tiff_path = sink_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(sink_zarr_path, tiff_path, num_workers=24)

Loading z 0 - 200


100%|██████████| 200/200 [00:17<00:00, 11.75it/s]


Loading z 200 - 400


100%|██████████| 200/200 [00:15<00:00, 12.68it/s]


Loading z 400 - 600


...

100%|██████████| 200/200 [00:15<00:00, 12.52it/s]


Loading z 1600 - 1800


100%|██████████| 200/200 [00:16<00:00, 11.91it/s]


Loading z 1800 - 2000


100%|██████████| 200/200 [00:17<00:00, 11.47it/s]


### (2)

In [9]:
# (2)
grid_path = '/mnt/share3/webster/mEhmAD_2-3/warping_grids/2-lec_grid_flatten_upsampled.npy'
original_shape = (16400,7816,5380)
new_grid_spacing = 3*(64,) # doesn't matter if range_mode= 'moving'
range_mode='moving'

# We will compute  the grid spacing from the zarr specifically 
save_grid_path = '/mnt/share3/webster/mEhmAD_2-3/warping_grids/2-lec_grid_flatten_upsampled_cropped_abeta.npy'

# these are the coordinates in original image of FOV
xrange = [4554,7420]
yrange = [5177,6742]
zrange = [0,1940]

################
grid_new,new_shape = crop_grid(grid_path, original_shape, new_grid_spacing, range_mode=range_mode, 
                         xrange=xrange, yrange=yrange, zrange=zrange, save_grid_path=save_grid_path)

New fixed image shape: (2959, 1744, 2010)
New fixed range: x 4501 7460
New fixed range: y 5103 6847
New fixed range: z 129 2139


In [10]:
# 2
fixed_zarr_path = new_shape
moving_zarr_path = bdir(name_prefix2+'.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(64,)
chunks=3*(200,)
nb_workers = 8


# grid I/O 
save_grid_values_path = None
use_grid_values_path = '/mnt/share3/webster/mEhmAD_2-3/warping_grids/2-lec_grid_flatten_upsampled_cropped_abeta.npy'




##########################

moving_pts_paths = None # doesn't actually matter for this 
fixed_pts_paths =  None

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None
zadd = fixed_zarr_path[2]-zarr.open(moving_zarr_path).shape[2]

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,zadd=zadd,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
# tiff_path = warped_zarr_path[:-5]+'_tiffs'
# convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(2959, 1744, 2010)
Loading grid values...
Warping image...
Moving image size: 17.4029252 GB


100%|██████████| 1485/1485 [02:41<00:00,  9.17it/s]


Time elapsed: 4.939342 minutes


## TPS warp using ptau grid

[Return to top](#top)

<a id='anchorwarp'></a>

New fixed range: x 4501 7460<br>
New fixed range: y 5103 6847<br>
New fixed range: z 129 2139<br>
    
### These are the coordinates of the original 
xrange = [4545,7353]<br>
yrange = [5226,6718]<br>
zrange = [0,2000]<br>


In [11]:
# We need to crop the flattened to be the same size as the ptau flattened zarr to be warped

# bounding box of moving in flat frame
xrange = [4545-4501,7353-4501]
yrange = [5226-5103,6718-5103]
zrange = [0,2000-129]
num_workers = 24
source_zarr_path = bdir(name_prefix2+'_flattened.zarr')
sink_zarr_path = bdir(name_prefix2+'_flattened_recropped.zarr')



###############
crop_zarr(source_zarr_path, sink_zarr_path, xrange=xrange, yrange=yrange, zrange=zrange, 
          load_num_slices=None, num_workers=num_workers)

Processing chunk x:44-2852, y:123-1615, z:0-200


100%|██████████| 120/120 [00:02<00:00, 44.32it/s]


Processing chunk x:44-2852, y:123-1615, z:200-400


100%|██████████| 120/120 [00:07<00:00, 15.77it/s]


Processing chunk x:44-2852, y:123-1615, z:400-600


...

100%|██████████| 120/120 [00:07<00:00, 15.49it/s]


Processing chunk x:44-2852, y:123-1615, z:1600-1800


100%|██████████| 120/120 [00:07<00:00, 15.53it/s]


Processing chunk x:44-2852, y:123-1615, z:1800-1871


100%|██████████| 120/120 [00:03<00:00, 36.66it/s]


In [12]:
# (2)
grid_path = '/mnt/share3/webster/mEhmAD_2-3/warping_grids/grid_ptauwarp_cropped_r2.npy'
original_shape = (2651, 1347, 2000)
new_grid_spacing = 3*(64,) # doesn't matter if range_mode= 'moving'
range_mode='moving'

# We will compute  the grid spacing from the zarr specifically 
save_grid_path = '/mnt/share3/webster/mEhmAD_2-3/warping_grids/grid_ptauwarp_cropped_r2_abeta.npy'

# these are the coordinates in original image of FOV
xrange = [0,2808]
yrange = [0,1492]
zrange = [129,2000]

################
grid_new,new_shape = crop_grid(grid_path, original_shape, new_grid_spacing, range_mode=range_mode, 
                         xrange=xrange, yrange=yrange, zrange=zrange, save_grid_path=save_grid_path)

New fixed image shape: (2651, 1347, 1822)
New fixed range: x 0 2651
New fixed range: y 0 1347
New fixed range: z 0 1822


In [13]:
# 2
fixed_zarr_path = new_shape #(2651, 1347, 2000)
moving_zarr_path = bdir(name_prefix2+'_flattened_recropped.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened_recropped_ptauwarp_r2.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(16,)
chunks=3*(200,)
nb_workers = 8


# grid I/O 
save_grid_values_path = None
use_grid_values_path = '/mnt/share3/webster/mEhmAD_2-3/warping_grids/grid_ptauwarp_cropped_r2_abeta.npy'




##########################

moving_pts_paths = None # doesn't actually matter for this 
fixed_pts_paths =  None

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None
zadd = fixed_zarr_path[2]-zarr.open(moving_zarr_path).shape[2]

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,zadd=zadd,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(2651, 1347, 1822)
Loading grid values...
Warping image...
Moving image size: 15.677243712 GB


100%|██████████| 980/980 [01:58<00:00,  8.26it/s]


Time elapsed: 4.015730 minutes
Loading z 0 - 200


100%|██████████| 200/200 [00:23<00:00,  8.56it/s]


Loading z 200 - 400


...

100%|██████████| 200/200 [00:19<00:00,  8.29it/s]


Loading z 1600 - 1800


100%|██████████| 200/200 [00:22<00:00,  8.85it/s]


Loading z 1800 - 1822


100%|██████████| 22/22 [00:02<00:00,  8.39it/s]
